In [0]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('dataset.zip')  # Save Drive file as a local file

dataset.zip


In [0]:
import pandas as pd
import numpy as np
import os
import keras
from pathlib import Path
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
# !unzip dataset.zip -d ./

images = []
labels = []
p = Path('dataset')
dirs = p.glob('*')
total = 0
for folder in dirs:
    label = str(folder).split('/')[-1]
    cnt = 0
    print(label)
    # read png file 
    for img_path in folder.glob('*.png'):
        img = image.load_img(img_path, target_size=(96, 96))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)
        cnt += 1
        total += 1
    
    # read jpg files
    for img_path in folder.glob('*.jpg'):
        img = image.load_img(img_path, target_size=(96, 96))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)
        cnt += 1
        total += 1
        
print('There are' ,total,"images")

Mewtwo


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Clefairy
Pidgeotto
Articuno
Rapidash
Sandshrew
Nidoran♂
Weepinbell
Rattata
Krabby
Nidorino
Zapdos
Beedrill
Muk
Omanyte
Venomoth
Vileplume
Arcanine
Rhyhorn
Kakuna
Horsea
Diglett
Metapod
Nidoqueen
Kangaskhan
Magikarp
Charmeleon
Butterfree
MrMime
Kabuto
Hitmonchan
Dragonite
Moltres
Jynx
Gloom
Tentacool
Flareon
Marowak
Drowzee
Nidorina
Mankey
Eevee
Goldeen
Machop
Magneton
Electabuzz
Victreebel
Wigglytuff
Cloyster
Raichu
Parasect
Cubone
Mew
Psyduck
Poliwag
Weezing
Scyther
Aerodactyl
Kabutops
Arbok
Chansey
Nidoran♀
Koffing
Weedle
Bellsprout
Persian
Slowbro
Squirtle
Tentacruel
Pidgeot
Hypno
Seadra
Ditto
Paras
Nidoking
Machoke
Abra
Lickitung
Pidgey
Geodude
Pinsir
Dodrio
Meowth
Snorlax
Grimer
Charmander
Onix
Graveler
Gengar
Sandslash
Raticate
Fearow
Charizard
Golduck
Bulbasaur
Slowpoke
Kadabra
Dratini
Gyarados
Poliwrath
Clefable
Staryu
Blastoise
Electrode
Machamp
Dragonair
Venonat
Spearow
Tangela
Seel
Growlithe
Ivysaur
Primeape
Doduo
Rhydon
Vulpix
Starmie
Haunter
Voltorb
Kingler
Omastar
Gastly


In [0]:
data = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

In [0]:
input_shape = data[0].shape

base_model= VGG16(weights='imagenet',include_top=False, input_shape=input_shape)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(2048,activation='relu')(x) 
x=Dense(2048,activation='relu')(x) 
preds=Dense(149,activation='softmax')(x) 


model=Model(inputs=base_model.input,outputs=preds)

In [0]:
data.shape
from sklearn.utils import shuffle
data, labels = shuffle(data, labels, random_state=42)
labels

array(['Metapod', 'Mankey', 'Nidoqueen', ..., 'Pinsir', 'Nidorino',
       'Staryu'], dtype='<U10')

In [0]:
labels.shape
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
for layer in model.layers:
    layer.trainable=False

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2)

In [0]:
EPOCHS = 10
INIT_LR = 1e-3
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS), metrics=["accuracy"])


H = model.fit(x=trainX, y=trainY,validation_data=(testX, testY), epochs=EPOCHS, verbose=1, batch_size=128)

Train on 8499 samples, validate on 2125 samples
Epoch 1/10
8499/8499 [==============================] - 1071s 126ms/step - loss: 4.5395 - acc: 0.0940 - val_loss: 3.9899 - val_acc: 0.1628
Epoch 2/10
8499/8499 [==============================] - 1070s 126ms/step - loss: 3.6115 - acc: 0.2136 - val_loss: 3.4144 - val_acc: 0.2532
Epoch 3/10
8499/8499 [==============================] - 1071s 126ms/step - loss: 3.0560 - acc: 0.3119 - val_loss: 3.1715 - val_acc: 0.2969
Epoch 4/10
8499/8499 [==============================] - 1068s 126ms/step - loss: 2.6665 - acc: 0.3768 - val_loss: 3.0454 - val_acc: 0.3252
Epoch 5/10
8499/8499 [==============================] - 1069s 126ms/step - loss: 2.3531 - acc: 0.4478 - val_loss: 2.8259 - val_acc: 0.3595
Epoch 6/10
8499/8499 [==============================] - 1068s 126ms/step - loss: 2.0768 - acc: 0.4942 - val_loss: 2.7796 - val_acc: 0.3689
Epoch 7/10
8499/8499 [==============================] - 1040s 122ms/step - loss: 1.8261 - acc: 0.5519 - val_loss: 2.77

In [0]:
m